https://github.com/anhthuan1999/Vietnamese-News-Classification/tree/master/models
https://github.com/dangvansam/phobert-text-classification/blob/main/train_classifer_keras.py
https://www.kaggle.com/code/siddharth63/multilabel-text-classification/notebook
https://huggingface.co/joeddav/xlm-roberta-large-xnli
https://viblo.asia/p/bert-roberta-phobert-bertweet-ung-dung-state-of-the-art-pre-trained-model-cho-bai-toan-phan-loai-van-ban-4P856PEWZY3



In [1]:
import pandas as pd
nRowsRead = 1000  # specify 'None' if want to read whole file
# news_5000_19032020_201604.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./data/news_labeled_title.csv',
                  delimiter=',', nrows=nRowsRead)
df1.dataframeName = 'news_labeled.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 3 columns


In [2]:
# Preparing train data

train_df_f = df1[['title', 'Labels']]
train_df_f.columns = ['text', 'labels']
train_df = train_df_f
print(train_df.head())


def convert_to_list(x):
    # Remove the square brackets from the string
    list_string = x.strip("[]")

    # Split the string into individual elements
    list_elements = list_string.split()

    # Convert the elements to integers and create the final list
    list_object = [int(element) for element in list_elements]
    return list_object


train_df['labels'] = train_df['labels'].apply(convert_to_list)

print('type', type(train_df['labels'][0]))

                                          text  \
0    Ngừng lấy mẫu xét nghiệm nCoV tại Nội Bài   
1                    Số ca nghi nhiễm giảm nhẹ   
2  Thuốc cúm Nhật hiệu quả rõ rệt trị Covid-19   
3       Loại thuốc tiềm năng điều trị Covid-19   
4   Bác sĩ châu Âu đang lặp lại sai lầm Vũ Hán   

                                              labels  
0  [1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0...  
1  [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...  
2  [1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...  
3  [0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...  
4  [0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...  
type <class 'list'>


/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_65680/1899247606.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['labels'] = train_df['labels'].apply(convert_to_list)


In [3]:
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens

In [4]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [10]:
from torch.nn.functional import pad
from underthesea import classify, word_tokenize, sent_tokenize
from pickle import load

class MultiClassify:
    max_len = 400

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        # Initialize the multi-output classifier (using Support Vector Machine classifier here)
        self.classifier = MultiOutputClassifier(KNeighborsClassifier())

    def trim_words(sentence, num_word=200):
        # Split the sentence into words
        words = sentence.split()
        # Trim the sentence to 5 words
        trimmed_sentence = " ".join(words[:num_word])
        return trimmed_sentence

    def extract_hidden_state(self, outputs):
        hiddenSize = outputs[0].size()[0]
        # flatten dimension 1 and 2, and hold batch dimension 0
        embeddings = outputs[0].view(hiddenSize, -1).numpy()
        return embeddings

    def fit(self, X_train, Y_train):
        X_train_vec = self.vectorize_mean(X_train)
        # Train the classifier
        self.classifier.fit(X_train_vec, Y_train)

        return self
    
    def vectorize_mean(self, sentences):
        encodedText = tokenizer.batch_encode_plus(
            sentences, padding=True, truncation=True, max_length=300)['input_ids']
        embeddedTitle = self.model(torch.tensor(encodedText))
        with torch.no_grad():
            print('hiddenState', self.extract_hidden_state(embeddedTitle).shape)

        embedSqueeze = embeddedTitle['last_hidden_state'].squeeze(0)
        print('embedSqueeze', embedSqueeze.shape)
        embedMean = embedSqueeze.mean(axis=1)
        return embedMean.detach().numpy()

    def loadPreTrainModel(self):
        self.mlb = load(open('./models/mlb.pkl', 'rb'))
        print(self.mlb.classes_)
        self.keyword_clusters = load(open('./models/keyword_clusters.pkl', 'rb'))
        print(self.keyword_clusters)
        self.kmeans = load(open('./models/kmeans.pkl', 'rb'))
        return self

    def predict(self, X_test):
        # Make predictions on the test data
        X_test = self.vectorize_mean(X_test)

        predictions = self.classifier.predict(X_test)
        print(predictions, len(predictions[0]))

        res = self.mlb.inverse_transform(predictions)
        pred = []
        for t in res:
            pred.append(self.getKeywordByIndex(t))
        return pred

    def getKeywordByIndex(self, param):
        if isinstance(param, int):
            return self.keyword_clusters[param][:3]
        elif isinstance(param, list):
            return [self.keyword_clusters[p][:3] for p in param]
        elif isinstance(param, tuple):
            return [self.keyword_clusters[p][:3] for p in param]





classify = MultiClassify(phobert, tokenizer)
classify.loadPreTrainModel()

classify.fit(train_df['text'], train_df['labels'])


# train_df_check = train_df[:100]
# X_train = vectorize_arr(train_df_check['text'])
# y_train = train_df_check['labels'].values.tolist()
# print('y_train', np.array(y_train).shape, X_train.shape)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148]
{130: [' 101 tuổi', ' L.Casei 431', ' bệnh phổi', ' chơi thể thao', ' chống chọi', ' củ chuối', ' dị vật phổi', ' ghép phổi', ' hoa chuối', ' khoai tây nghiền', ' lối sống lành mạnh', ' lợi khuẩn', ' nốt ruồi', ' quai động mạch', ' stress cuối năm', ' ung thư phổi', ' xương đùi', ' đối phó dịch', ' đối phó stress', 'hoại tử chân', 'làm sạch phổi', 'lợi khuẩn', 'ung thư phổi'], 3: [' 14 ngày'

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:85                                                                                   │
│                                                                                                  │
│   82 classify = MultiClassify(phobert, tokenizer)                                                │
│   83 classify.loadPreTrainModel()                                                                │
│   84                                                                                             │
│ ❱ 85 classify.fit(train_df['text'], train_df['labels'])                                          │
│   86                                                                                             │
│   87                                                                                             │
│   88 # train_df_check = train_df[:100]                                                           │
│                                                                                                  │
│ in fit:30                                                                                        │
│                                                                                                  │
│   27 │   def fit(self, X_train, Y_train):                                                        │
│   28 │   │   X_train_vec = self.vectorize_mean(X_train)                                          │
│   29 │   │   # Train the classifier                                                              │
│ ❱ 30 │   │   self.classifier.fit(X_train_vec, Y_train)                                           │
│   31 │   │                                                                                       │
│   32 │   │   return self                                                                         │
│   33                                                                                             │
│                                                                                                  │
│ /Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/sklearn/multioutput.py:450 in   │
│ fit                                                                                              │
│                                                                                                  │
│    447 │   │   self : object                                                                     │
│    448 │   │   │   Returns a fitted instance.                                                    │
│    449 │   │   """                                                                               │
│ ❱  450 │   │   super().fit(X, Y, sample_weight, **fit_params)                                    │
│    451 │   │   self.classes_ = [estimator.classes_ for estimator in self.estimators_]            │
│    452 │   │   return self                                                                       │
│    453                                                                                           │
│                                                                                                  │
│ /Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/sklearn/multioutput.py:201 in   │
│ fit                                                                                              │
│                                                                                                  │
│    198 │   │   y = self._validate_data(X="no_validation", y=y, multi_output=True)                │
│    199 │   │                                                                                     │
│    200 │   │   if is_classifier(self):                                                           │
│ ❱  201 │   │   │   check_classification_targets(y)                                               │
│    202 │   │                                               

In [ ]:
classify.loadPreTrainModel()

classify.fit(train_df['text'], train_df['labels'])


In [6]:
from pickle import load
mlb = load(open('./models/mlb.pkl', 'rb'))
print(mlb.classes_)
keyword_clusters = load(open('./models/keyword_clusters.pkl', 'rb'))

print(keyword_clusters)

kmeans = load(open('./models/kmeans.pkl', 'rb'))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148]
{130: [' 101 tuổi', ' L.Casei 431', ' bệnh phổi', ' chơi thể thao', ' chống chọi', ' củ chuối', ' dị vật phổi', ' ghép phổi', ' hoa chuối', ' khoai tây nghiền', ' lối sống lành mạnh', ' lợi khuẩn', ' nốt ruồi', ' quai động mạch', ' stress cuối năm', ' ung thư phổi', ' xương đùi', ' đối phó dịch', ' đối phó stress', 'hoại tử chân', 'làm sạch phổi', 'lợi khuẩn', 'ung thư phổi'], 3: [' 14 ngày'

In [7]:
# Make predictions on the test data
X_test_texts = [
    'Số ca nghi nhiễm giảm nhẹ',
    'Thuốc cúm Nhật hiệu quả rõ rệt trị Covid-19',
]

X_test = vectorize_arr(X_test_texts)

predictions = classifier.predict(X_test)
print(predictions, len(predictions[0]))

res = mlb.inverse_transform(predictions)
print(res)
print(keyword_clusters[4])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4 │   'Thuốc cúm Nhật hiệu quả rõ rệt trị Covid-19',                                          │
│    5 ]                                                                                           │
│    6                                                                                             │
│ ❱  7 X_test = vectorize_arr(X_test_texts)                                                        │
│    8                                                                                             │
│    9 predictions = classifier.predict(X_test)                                                    │
│   10 print(predictions, len(predictions[0]))                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'vectorize_arr' is not defined



- mỗi cluster cần lấy ra 3 keyword có nhiều bài post nhất để đại diện 
- convert nhanh kết quả tới labels

In [8]:
def getKeywordByIndex(param):
    if isinstance(param, int):
        return keyword_clusters[param][:3]
    elif isinstance(param, list):
        return [keyword_clusters[p][:3] for p in param]
    elif isinstance(param, tuple):
        return [keyword_clusters[p][:3] for p in param]


for t in res:
    print("list keyword:", getKeywordByIndex(t))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │   │   return [keyword_clusters[p][:3] for p in param]                                     │
│    8                                                                                             │
│    9                                                                                             │
│ ❱ 10 for t in res:                                                                               │
│   11 │   print("list keyword:", getKeywordByIndex(t))                                            │
│   12                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'res' is not defined

In [9]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

X, y = make_multilabel_classification(
    n_classes=140, n_samples=1000, n_features=768, random_state=0)
print(X.shape, y)
clf = MultiOutputClassifier(KNeighborsClassifier()).fit(X, y)
clf.predict(X[-2:])

(1000, 768) [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]])